In [359]:
from rap_db import *
import re
from os import listdir
from os.path import isfile, join
import nltk
#nltk.download()
estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

In [365]:
#need this to figure out verses
cur = estconn.cursor()
cur.execute('''SELECT artist_name FROM artists;''')
quer = cur.fetchall()
MASTER_RAPPER_SET = set(art[0].lower() for art in quer)
#known aliases that aren't in there
aliases = {'em'}
MASTER_RAPPER_SET = MASTER_RAPPER_SET|aliases

create_music_tables(estconn, bypass = True)
bulk_load(estconn, ['mf_doom', 'chance_the_rapper'])

In [366]:
doom = basic_lyrc_pull(estconn, 'MF Doom')
chance = basic_lyrc_pull(estconn, 'Chance The Rapper')
em = basic_lyrc_pull(estconn, 'Eminem')

In [367]:
#need to find the best way to mark of important parts of the song,
# {**} == not sure but one thing is can mean someone saying something
# {?} == dont know line
# [Artist] == i think this is when an artist s speaking
# [yes] == can also mean like () whe someone says somthing, have to use [Capital First letter] to differentiate
#or a great way to differentiate is to check contents of [] against db of artists with tolower and edge case for chorus or outro or intro
# * == annotator notes
# (action!) == i think this is sound of action, can also be someone saying something in track
# () == someone saying something or repeat signal
# (**) ==
# looks like () and [] and {} are pretty much interchanchable and it matters a lot more whats inside
# [???] == i think it means they didn't know
# "" == i think this is when someone else is talking over the track
# Man1: denoting that someone is speaking in a conversation
#words = re.sub("([\(\[]).*?([\)\]])", "", lyrics)
#words = re.sub("\n","--",words)


In [379]:
class raw_lyrics():
    #currently written, some will pick each other up, might be ok, can fix later if needed
    #can totally consolidate this later if there's no benefit to differentiating, but cant have start with [ and end with ] or }

    #regex are dope
    regex_commands = {'[intro]':'\[intro\]',
                      '[outro]':'\[outro\]',
                      '[chorus]':'\[.*chorus.*\]',
                      '[verse]':'\[.*verse.*|.*bridge.*\]',
                      '[]':'\[(?!intro|outro|.*chorus.*|.*verse.*|.*bridge.*).*\]',
                      '{}':'\{(.*?)\}',
                      #'\n':'\n', will use later, too anoyting to check rn
                      #still pulling others, may come back to this later
                      #'**':'\*(.*?)\*',
                      '{**}':'\{\*(.*?)\*\}',
                      '()':'\((.*?)\)',
                      '""':'"(.*?)"',
                      '?':'\?+',
                      '*text':'[\n| ]\*[^\n|\{|\(|\[]*'}
                        #second part stops on \n,{,[, or (, might need to add/take away more exceptions
    
    def __init__(self, raw_text, name):
        self.raw_text = raw_text
        self.name = name
    
    #basically want to go through dic and use a re ex to pull out the matching phrase and mark the spot in the string
    #with the marks, I can cut up the lyrics accordingly, I use extras_set/extras_list to rank my segments in order
    def assign_extras(self):
        self.extras = dict(zip(raw_lyrics.regex_commands.keys(), [[]]*len(raw_lyrics.regex_commands)))
        extras_set = set()
        
        for ext, regex_command in raw_lyrics.regex_commands.items():
            finder = re.compile(regex_command)
            ap_list = []
            
            for match in finder.finditer(self.raw_text.lower()):
                #match has a start pos and a match value, add to list to append
                if match.group()[1:-1] in MASTER_RAPPER_SET:
                    #this if statement is a catch all for rappers names mentioned to be sent to verse
                    #wanna check things like '[eminem]' so [1:-1]
                    #its painstacking but I want verse as accurate as possible
                    self.extras['[verse]'].append((match.group(), (match.start(), match.start()+len(match.group()))))
                else:
                    ap_list.append((match.group(), (match.start(), match.start()+len(match.group()))))
                    
                extras_set = extras_set|{(match.group(), (match.start(), match.start()+len(match.group())))}
            self.extras[ext] = ap_list
            
        #gonna be used later for ranking purposes
        self.extras_list = sorted(list(extras_set), key=lambda x: x[1][0])
        
    #this removes any of the regex formations above and recalculates the new positions
    def remove_and_reass(self, rem = []):
        for r in rem:
            if r in raw_lyrics.regex_commands.keys():
                self.raw_text = re.sub(raw_lyrics.regex_commands[r], '', self.raw_text)
        self.assign_extras()
        
    

In [380]:
#all these have to be loaded with the possiblity of not existing
#looks like text after label corresponds

#gonna iterate through intro, outro, chorus and verse and segment accordingly

#NEVER HAVE THESE OBJECTS PERMANENTLY STORE THE FULL TEXT - too slow
#intro class to segment beginning
class text_block():
    def __init__(self, label, start, end, rank):
        self.label = label
        self.start = start
        self.end = end
        self.rank = rank
    def text_segs(self, raw_text, after):
        self.content = raw_text[self.end:after]

#here, before should be the previous objects end 
class verse(text_block):
    def text_segs(self, raw_text, after):
        self.pre_text = raw_text[self.end:after]

In [382]:
song_list = []
for art, text in {**em, **doom, **chance}.items():
    song = raw_lyrics(text, art)
    song.assign_extras()
    song.remove_and_reass(['*text', '?'])
    song_list.append(song)

What you did last time:

made ranking system for extras built into raw_lyrics class
found a way to id verses, choruses, intro, outro, etc. via regex and list checking
made inital class to hold each lyrics type

next:
feed in sorted list to make each block_text class type, all of those will be held in a song class
now lyrics will have type
play aroudn with colorizing lyrics based on type for quick texting

goals:
continue to clean
build a good and anti edge case foundation so it's easy to keep builiding up
lots of cleaning on comments and uncessary shit

I think we're getting close!

In [371]:
estconn.close()